# Notebook for å teste azure search funksjonalitet

In [ ]:
from datetime import datetime
from itertools import batched

from nks_kbs_analyse.azure_search import create_store
from nks_kbs_analyse.knowledgebase import clean_doc, load, split_documents

## Hent dokumenter fra BigQuery

In [ ]:
# Hent dokumenter fra kunnskapsbasen

## merk - før denne kjøringen kommenterte vi ut en del kolonner
## (beholdt kun de vi har eksisterende testspørsmål for)
docs = list(load())
num_docs = len(docs)

In [ ]:
# Fjerne div. overflødige newline-tegn
cleaned_docs = [clean_doc(doc) for doc in docs]

Fikser litt på metadataene:

In [ ]:
# Legg til opplastingtidspunkt på alle dokumenter
today = datetime.today().isoformat()
for doc in cleaned_docs:
    doc.metadata["EmbeddingCreation"] = today
# azure-implementasjon har foreløpig kun støtte for string-metadata
for doc in cleaned_docs:
    for k, v in doc.metadata.items():
        if isinstance(v, datetime):
            v = v.isoformat()
        elif isinstance(v, int):
            v = str(v)
        doc.metadata[k] = v

## Chunk size 1500, overlap 100

In [ ]:
# husk å sette riktig navn på search-instans i settings.py før du kjører denne
store = create_store()

In [ ]:
splits = split_documents(cleaned_docs, chunk_size=1500, overlap=100)
print("Laster opp dokumenter")
# batcher opplastingen av dokumentene
ids = []
for batch in batched(splits, 500):
    batch_ids = store.add_documents(batch)
    ids.extend(batch_ids)
print(f"Lastet opp {num_docs} dokumenter, totalt {len(ids)} tekster")

## Chunk size 4000, overlap 200

In [ ]:
# husk å sette riktig navn på search-instans i settings.py før du kjører denne
store = create_store()

In [ ]:
splits = split_documents(cleaned_docs, chunk_size=4000, overlap=200)
print("Laster opp dokumenter")
# batcher opplastingen av dokumentene
ids = []
for batch in batched(splits, 500):
    batch_ids = store.add_documents(batch)
    ids.extend(batch_ids)
print(f"Lastet opp {num_docs} dokumenter, totalt {len(ids)} tekster")

## For å tømme en indeks

In [ ]:
# store.clear()